# PURSUE Python for HEP: Looking At Real Data

## Loading the Data

* Uproot can open not just local ROOT files, but it can also open remote files. We will be working with a file which is hosted by CERN's Open Data Portal.
* Note: If you are unable to access the remote file, uncomment the next cell and run it to download an alternative, smaller file.

In [ ]:
# !wget https://github.com/masonproffitt/uproot-tutorial-notebooks/raw/master/uproot-tutorial-file.root

In [ ]:
import uproot

file = uproot.open(
    "root://eospublic.cern.ch//eos/opendata/cms/derived-data/AOD2NanoAODOutreachTool/Run2012BC_DoubleMuParked_Muons.root"
    # "./uproot-tutorial-file.root"
)
file.classnames()

* Accesing the tree as such just provides you a handle for it. 

In [ ]:
tree = file["Events"]
tree

* So we use the `.array()` method to access the actual data. However, the amount of data in this file is large, so we can add arguments to this method to limit the amount of entries we read, thus reducing load times.

In [ ]:
tree.show()

In [ ]:
tree["nMuon"].array(entry_start=0, entry_stop=100)

* To read more than just a single TBranch, we can also specify the particular branches we want to load in the `array()` method.
* When we do this, we get an an Awkward array which contains Awkward records
  * Awkward array: Awkward equivalent of a Numpy array
  * Awkward record: Awkward equivalent of a dictionary
* The true power of awkward arrays becomes evident when it contains records. In this situation, you can pass a key value in between brakets to the *array*, and you will get back an array containing the value for that key for every record in that array.

In [ ]:
muons_npt = tree.arrays(
    ["nMuon", "Muon_pt"], entry_stop=1000
)
print(type(muons_npt))
print(type(muons_npt[0]))
muons_npt

In [ ]:
muons_npt["Muon_pt"]

* **This lets us completely shift how we approach problems which might otherwise be cumbersome to deal with while doing analysis with HEP data: by having data structures that accomodate jagged arrays so well, we can adopt a fully array-oriented approach to our programming, allowing us to take advantage of vectorized operations at every step, simplfying the paralellization of our algorithms and helping us avoid slow loops.**

**Exercise**: Print out the pt and charge of the muons from events 100 to 115. Do this in two different ways, but only using Awkward array slicing+filtering. Once solved, determine which one of these ways is faster by using the `%%timeit` magic. Why do you think one is faster than the other?

In [ ]:
%%timeit
muons_npt[100:115]["Muon_pt"]

In [ ]:
%%timeit
muons_npt["Muon_pt"][100:115]

* Given that the data we typically deal with is huge, unless neccesary, its good practice to specify the particular TBranches you want instead of doing something like `tree.arrays()` as this will load in everything, including things you might not be interested in.
* In addition to the way shown before, you can also get a subset of the branches by passing a `filter_name` argument to `tree.arrays()`.

In [ ]:
tree.arrays(filter_name="Muon_*", entry_stop=100)

* If you don't know what the name of the available branches are and you don't want to print them all, you can search for them using the following syntax.

In [ ]:
tree.keys(filter_name="Muon_*")

* You can use the same type of filtering when calling `arrays()`

In [ ]:
tree.arrays(filter_name="Muon_*", entry_stop=100)

## Exploring the Data

In [ ]:
# Loading in some data from all branches
muons = tree.arrays(entry_stop=1500)
muons

* Accoring to the above cell output, each events in this dataset have 6 total "attributes":
  * `nMuon`: The number of muons in the event
  * `Muon_pt`: The $p_T$ of each of those muons
  * `Muon_eta`: The $\eta$ coordinate of each muon
  * `Muon_phi`: The $\phi$ coordinate of each muon
  * `Muon_mass`: The mass of each muon
  * `Muon_charge`: The charge of each muon
* Here, `Muon_phi` and `Muon_eta` refers to the corresponding spacial coordinates used in the CMS coordinate system.

<div style="text-align: center;">
  <img src="./assets/axis3D_CMS.png" alt="roottree" style="width: 600px"/>
</div>

* We can take a quick look at a small sample of the data to get an idea of how it is structured in the following way. Note that this is not the usual way to do it is simply for pedagogical purposes.

In [ ]:
import pandas as pd

tree.arrays(library="pd", entry_stop=10)

* Knowing what is in this data, we can go ahead and start doing something with it. Let's begin by constructing the masses. To do this, we can use a feature that exists for Awkward arrays that we saw in Numpy arrays: filtering through a mask array.

In [ ]:
# If an event has 2 muons, its corresponding element in the mask is True. False otherwise.
mask = muons["nMuon"] == 2
mask

In [ ]:
# Only keeping di-muon events
dimuons = muons[mask]
dimuons

In [ ]:
# Getting the eta, pt and phi of each of the muons in the dimuon events
pt0 = dimuons["Muon_pt"][:,0]
pt1 = dimuons["Muon_pt"][:,1]
eta0 = dimuons["Muon_eta"][:,0]
eta1 = dimuons["Muon_eta"][:,1]
phi0 = dimuons["Muon_phi"][:,0]
phi1 = dimuons["Muon_phi"][:,1]

* The mass is computed using the following equation

$$
m_{\mu\mu} = \sqrt{(E_1 + E_2)^2 - \|\textbf{p}_1 + \textbf{p}_2 \| ^2} = \sqrt{(E_1+E_2)^2-((p_{1_x}+p_{2_x})^2+(p_{1_y}+p_{2_y})^2+(p_{1_z}+p_{2_z})^2)}|
$$

* If we have that $E >> m$, then we can rewrite this as:
$$
m_{\mu\mu} = \sqrt{
    2p_{T,0} p_{T,1} * \left(\cosh(\eta_0 - \eta_1) - \cos(\phi_0- \phi_1)\right)
}
$$

In [ ]:
import numpy as np
mass = np.sqrt(2 * pt0 * pt1 * (np.cosh(eta0 - eta1) - np.cos(phi0 - phi1)))
mass

* We now use the Hist library, one of the many tools offered by Scikit-HEP. This library offers a user-friendly API for the Boost-histogram Python library which offers a highly efficient and flexible histogramming framework. Some example features are:
  * Built on the C++ Boost.Histogram libary, so offers good performance
  * Flexibility: supports multiple types of histograms
  * N-dimensional histograms
  * Axis customization
  * Named axes
  * Built-in support for plotting histograms using Matplotlib
  * Integrates well with numpy and pandas
* Making a histogram for the dimuon mass.

([Hist docs](https://hist.readthedocs.io/en/latest/user-guide/axes.html))

In [ ]:
import hist
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use("CMS")

In [ ]:
# Using Hist + Matplotlib
fig, ax = plt.subplots()

masshist = hist.Hist(
    hist.axis.Regular(120, 0, 200, label="mass [GeV]", name="mass"),
)
masshist.fill(mass)
masshist.plot1d(ax=ax, label="Dimuon mass [GeV]", color="red")

ax.set_xlabel("Mass [GeV]")
ax.set_ylabel("Count")
ax.set_title("Dimuon Mass Distribution")
ax.legend()

plt.show()

* In the above plot, you can clearly see a peak around 90 GeV. This corresponds to the mass of the Z bosons and shows that we have been able to succesfully reconstruct its mass! However, what we have done so far is rather rudimentary. Firslty, we got rid of a bunch of events which have more than 2 muons, but which might also have 2 (or more) muons which came from Z. Also, we didn't make sure that quantities such as the charge were conserved. We just added the masses of the muons and hoped for the best. We will deal with these sorts of consdierations later on.
* Before we continue, lets pause and see the whole code for procuding out plot.

In [ ]:
muons = tree.arrays(filter_name="/nMuon|Muon_(phi|eta|pt)/", entry_stop=100000)
dimuons_cut = muons["nMuon"] == 2
dimuons = muons[dimuons_cut]

pt0 = dimuons["Muon_pt"][:,0]
pt1 = dimuons["Muon_pt"][:,1]
eta0 = dimuons["Muon_eta"][:,0]
eta1 = dimuons["Muon_eta"][:,1]
phi0 = dimuons["Muon_phi"][:,0]
phi1 = dimuons["Muon_phi"][:,1]

mass = np.sqrt(2 * pt0 * pt1 * (np.cosh(eta0 - eta1) - np.cos(phi0 - phi1)))


In [ ]:
fig, ax = plt.subplots()

masshist = hist.Hist(hist.axis.Regular(120, 0, 120, label="mass [GeV]"))
masshist.fill(mass)
masshist.plot1d(ax=ax)

ax.set_title("Dimuon Mass")
ax.set_xlabel("$m_{\mu\mu}$")
ax.set_ylabel("Count")

plt.show()

* We will be focusing on Z and H for the most part. However, you might wonder what those extra peaks are from to the left of the Z peak. If we make the plot logarithmic, we'll be able to appreciate what going on there...

In [ ]:
fig, ax = plt.subplots()
hist_data = hist.Hist(
    hist.axis.Variable(np.logspace(np.log10(0.1), np.log10(1000), 100), name="mass", label="Invariant Mass [GeV]")
)
hist_data.fill(mass=mass)
hist_data.plot1d(ax=ax, histtype='step', color="red", linewidth=0.75)

ax.set_xscale("log")
ax.set_yscale("log")
ax.set_xlabel("Invariant Mass [GeV]")
ax.set_ylabel("Counts")
ax.set_title("Dimuon Invariant Mass Distribution")

plt.show()

* Compare the previous plot with this one...
  
<div style="text-align: center;">
  <img src="./assets/dimuonspectrumplt.png" alt="roottree" style="width: 400px"/>
</div>